## [ 문제 해결 방법 ]  
   
   
* train, test set을 load 한 뒤, baseline 먼저 잡기 (baseline에는 날짜컬럼 만지지 않기)   
* 본 문제에 가장 적합한 모델 선택하기 (RF, XGBoost, CatBoost 등)  
* 날짜컬럼 등 데이터 전처리 추가하기

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sf-crime/train.csv.zip
/kaggle/input/sf-crime/sampleSubmission.csv.zip
/kaggle/input/sf-crime/test.csv.zip


In [2]:
train = pd.read_csv("/kaggle/input/sf-crime/train.csv.zip")
test = pd.read_csv("/kaggle/input/sf-crime/test.csv.zip")

all_data = pd.concat([train, test])
all_data

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y,Id
0,2015-05-13 23:53:00,WARRANTS,WARRANT ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599,NaN
1,2015-05-13 23:53:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599,NaN
2,2015-05-13 23:33:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",VANNESS AV / GREENWICH ST,-122.424363,37.800414,NaN
3,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,NORTHERN,NONE,1500 Block of LOMBARD ST,-122.426995,37.800873,NaN
4,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,PARK,NONE,100 Block of BRODERICK ST,-122.438738,37.771541,NaN
...,...,...,...,...,...,...,...,...,...,...
884257,2003-01-01 00:01:00,NaN,NaN,Wednesday,MISSION,NaN,2600 Block of BRYANT ST,-122.408983,37.751987,884257.0
884258,2003-01-01 00:01:00,NaN,NaN,Wednesday,NORTHERN,NaN,1900 Block of WASHINGTON ST,-122.425342,37.792681,884258.0
884259,2003-01-01 00:01:00,NaN,NaN,Wednesday,INGLESIDE,NaN,5500 Block of MISSION ST,-122.445418,37.712075,884259.0
884260,2003-01-01 00:01:00,NaN,NaN,Wednesday,BAYVIEW,NaN,1500 Block of HUDSON AV,-122.387394,37.739479,884260.0


* 날짜 컬럼 처리

In [3]:
all_data["Dates"] = pd.to_datetime(all_data["Dates"])
all_data["year"] = all_data["Dates"].dt.year
all_data["month"] = all_data["Dates"].dt.month
all_data["day"] = all_data["Dates"].dt.day
all_data["hour"] = all_data["Dates"].dt.hour
all_data["minute"] = all_data["Dates"].dt.minute
all_data

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y,Id,year,month,day,hour,minute
0,2015-05-13 23:53:00,WARRANTS,WARRANT ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599,NaN,2015,5,13,23,53
1,2015-05-13 23:53:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599,NaN,2015,5,13,23,53
2,2015-05-13 23:33:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",VANNESS AV / GREENWICH ST,-122.424363,37.800414,NaN,2015,5,13,23,33
3,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,NORTHERN,NONE,1500 Block of LOMBARD ST,-122.426995,37.800873,NaN,2015,5,13,23,30
4,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,PARK,NONE,100 Block of BRODERICK ST,-122.438738,37.771541,NaN,2015,5,13,23,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
884257,2003-01-01 00:01:00,NaN,NaN,Wednesday,MISSION,NaN,2600 Block of BRYANT ST,-122.408983,37.751987,884257.0,2003,1,1,0,1
884258,2003-01-01 00:01:00,NaN,NaN,Wednesday,NORTHERN,NaN,1900 Block of WASHINGTON ST,-122.425342,37.792681,884258.0,2003,1,1,0,1
884259,2003-01-01 00:01:00,NaN,NaN,Wednesday,INGLESIDE,NaN,5500 Block of MISSION ST,-122.445418,37.712075,884259.0,2003,1,1,0,1
884260,2003-01-01 00:01:00,NaN,NaN,Wednesday,BAYVIEW,NaN,1500 Block of HUDSON AV,-122.387394,37.739479,884260.0,2003,1,1,0,1


* train, test 합치고 필요 없는 컬럼 삭제

In [4]:
all_data2 = all_data.drop(columns=["Id", "Category", "Descript", "Resolution", "Dates"])
all_data2

,DayOfWeek,PdDistrict,Address,X,Y,year,month,day,hour,minute
0,Wednesday,NORTHERN,OAK ST / LAGUNA ST,-122.425892,37.774599,2015,5,13,23,53
1,Wednesday,NORTHERN,OAK ST / LAGUNA ST,-122.425892,37.774599,2015,5,13,23,53
2,Wednesday,NORTHERN,VANNESS AV / GREENWICH ST,-122.424363,37.800414,2015,5,13,23,33
3,Wednesday,NORTHERN,1500 Block of LOMBARD ST,-122.426995,37.800873,2015,5,13,23,30
4,Wednesday,PARK,100 Block of BRODERICK ST,-122.438738,37.771541,2015,5,13,23,30
...,...,...,...,...,...,...,...,...,...,...
884257,Wednesday,MISSION,2600 Block of BRYANT ST,-122.408983,37.751987,2003,1,1,0,1
884258,Wednesday,NORTHERN,1900 Block of WASHINGTON ST,-122.425342,37.792681,2003,1,1,0,1
884259,Wednesday,INGLESIDE,5500 Block of MISSION ST,-122.445418,37.712075,2003,1,1,0,1
884260,Wednesday,BAYVIEW,1500 Block of HUDSON AV,-122.387394,37.739479,2003,1,1,0,1


* 문자열 전체 처리

In [5]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

c = all_data2.columns[all_data2.dtypes==object]

for i in c:
    all_data2[i] = le.fit_transform(list(all_data2[i]))
    
all_data2

,DayOfWeek,PdDistrict,Address,X,Y,year,month,day,hour,minute
0,6,4,20895,-122.425892,37.774599,2015,5,13,23,53
1,6,4,20895,-122.425892,37.774599,2015,5,13,23,53
2,6,4,24169,-122.424363,37.800414,2015,5,13,23,33
3,6,4,4418,-122.426995,37.800873,2015,5,13,23,30
4,6,5,1923,-122.438738,37.771541,2015,5,13,23,30
...,...,...,...,...,...,...,...,...,...,...
884257,6,3,7919,-122.408983,37.751987,2003,1,1,0,1
884258,6,4,5596,-122.425342,37.792681,2003,1,1,0,1
884259,6,2,11612,-122.445418,37.712075,2003,1,1,0,1
884260,6,0,4393,-122.387394,37.739479,2003,1,1,0,1


* 결측지 처리 (숫자형 컬럼)

In [6]:
all_data2 = all_data2.fillna(-1)

* all_data2를 train2, test2로 나누기

In [7]:
train2 = all_data2[:len(train)]
test2 = all_data2[len(train):]

* 평가 set 만들기 (모델 평가 위해)   

- 기존 train set 중 80%만 train set에 유지시키고, 20%는 test set으로 만들어서 평가

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_valid, Y_train, Y_valid = train_test_split(train2, train["Category"], test_size=0.2)
# train_test_split은 랜덤하게 데이터를 추출함 (random_state 같은 것으로 랜덤성 꼭 고려해 줘야함!)
X_train

,DayOfWeek,PdDistrict,Address,X,Y,year,month,day,hour,minute
619109,3,6,9797,-122.499342,37.781297,2006,7,30,0,1
76677,3,7,12820,-122.406691,37.785207,2014,5,4,14,16
31885,6,9,8869,-122.414406,37.784191,2014,12,10,18,55
262148,3,7,12681,-122.403405,37.775421,2011,10,23,19,29
637594,3,8,6025,-122.475663,37.717141,2006,4,23,23,0
...,...,...,...,...,...,...,...,...,...,...
242661,6,0,20678,-122.390417,37.735593,2012,2,8,19,1
603009,2,3,20234,-122.419672,37.765050,2006,10,21,19,30
138056,2,7,20329,-122.419649,37.771006,2013,7,13,19,20
205229,4,5,754,-122.432228,37.766784,2012,8,23,4,30


In [9]:
X_valid

,DayOfWeek,PdDistrict,Address,X,Y,year,month,day,hour,minute
789146,1,3,16735,-122.411210,37.762974,2004,3,15,20,20
288382,4,7,12681,-122.403405,37.775421,2011,6,2,23,0
630335,2,7,8506,-122.389595,37.779861,2006,6,3,16,0
606538,4,9,1630,-122.414312,37.779834,2006,10,5,17,20
344915,3,8,8409,-122.493610,37.760913,2010,8,1,0,1
...,...,...,...,...,...,...,...,...,...,...
102279,1,4,13292,-122.416294,37.777494,2014,1,6,8,14
861783,4,1,11815,-122.403905,37.794171,2003,3,20,17,0
247019,2,0,11908,-122.387834,37.713107,2012,1,14,13,30
220473,0,3,9105,-122.422098,37.767811,2012,6,1,17,15


* Modeling

In [10]:
# 본 데이터는 카테고리적 느낌인 칼럼이 많기 때문에, CatBoostClassifier 사용!
# CPU->4개까지 사용 가능 (복잡산 계산 시 사용) 
# GPU->저성능 CPU를 매우 많은 개수 사용 (간단한 계산 시 사용)
# 본 문제는 간단한 계산을 수행하기 때문에, GPU 사용할 것임 (예측 시간 단축 위해)
from catboost import CatBoostClassifier
cbc = CatBoostClassifier(task_type="GPU") 
# cbc.fit(train2, train["Category"])
cbc.fit(X_train, Y_train, eval_set=(X_valid, Y_valid))    # BestTest 값이 ReaderBoard 점수랑 비슷!
result_c = cbc.predict_proba(test2)
result_c

Learning rate set to 0.203863
0:	learn: 3.0195654	test: 3.0189397	best: 3.0189397 (0)	total: 82.4ms	remaining: 1m 22s
1:	learn: 2.8466792	test: 2.8467711	best: 2.8467711 (1)	total: 161ms	remaining: 1m 20s
2:	learn: 2.7320372	test: 2.7323871	best: 2.7323871 (2)	total: 229ms	remaining: 1m 16s
3:	learn: 2.6550177	test: 2.6553741	best: 2.6553741 (3)	total: 300ms	remaining: 1m 14s
4:	learn: 2.6025810	test: 2.6030453	best: 2.6030453 (4)	total: 380ms	remaining: 1m 15s
5:	learn: 2.5625052	test: 2.5634079	best: 2.5634079 (5)	total: 452ms	remaining: 1m 14s
6:	learn: 2.5343966	test: 2.5354799	best: 2.5354799 (6)	total: 519ms	remaining: 1m 13s
7:	learn: 2.5117823	test: 2.5131205	best: 2.5131205 (7)	total: 587ms	remaining: 1m 12s
8:	learn: 2.4918566	test: 2.4936601	best: 2.4936601 (8)	total: 654ms	remaining: 1m 11s
9:	learn: 2.4766492	test: 2.4785909	best: 2.4785909 (9)	total: 725ms	remaining: 1m 11s
10:	learn: 2.4619264	test: 2.4642961	best: 2.4642961 (10)	total: 794ms	remaining: 1m 11s
11:	learn:

array([[6.24517728e-04, 5.35041787e-02, 6.51922955e-07, ...,
        1.30185539e-01, 4.40877298e-02, 1.94480135e-02],
       [1.49310670e-03, 8.63342429e-02, 9.09516437e-07, ...,
        7.34371855e-03, 5.95451154e-02, 4.95262729e-02],
       [1.35567575e-02, 1.29421423e-01, 1.68197404e-05, ...,
        4.00362227e-02, 2.52426530e-02, 4.81685653e-03],
       ...,
       [4.28768381e-03, 7.31917538e-02, 3.19337655e-04, ...,
        5.64803500e-02, 1.93032188e-02, 2.89590418e-03],
       [4.81911523e-04, 6.85941209e-02, 2.90125192e-03, ...,
        4.07129103e-02, 2.49291295e-02, 1.93103131e-03],
       [5.82573228e-04, 2.93641553e-02, 8.58760148e-04, ...,
        6.33137793e-02, 6.92285744e-03, 4.50485151e-04]])

* 예측 결과 확인 (정확도 확인)

* Save Result

In [11]:
sub = pd.read_csv("/kaggle/input/sf-crime/sampleSubmission.csv.zip")
sub

,Id,ARSON,ASSAULT,BAD CHECKS,BRIBERY,BURGLARY,DISORDERLY CONDUCT,DRIVING UNDER THE INFLUENCE,DRUG/NARCOTIC,DRUNKENNESS,...,SEX OFFENSES NON FORCIBLE,STOLEN PROPERTY,SUICIDE,SUSPICIOUS OCC,TREA,TRESPASS,VANDALISM,VEHICLE THEFT,WARRANTS,WEAPON LAWS
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
884257,884257,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
884258,884258,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
884259,884259,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
884260,884260,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [12]:
sub.iloc[:,1:] = result_c